In [ ]:
import ipywidgets as widgets
from IPython.display import Audio, display
import sounddevice as sd
import soundfile as sf
import numpy as np

# Function to record audio
def record_audio(duration=5, fs=44100):
    """
    Record audio for a given duration and sampling rate.
    """
    print(f"Recording for {duration} seconds...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=2)
    sd.wait()
    return recording, fs

# Button to handle recording
record_button = widgets.Button(description="Record Audio")

# Output widget to display audio widget
output = widgets.Output()

def on_record_button_clicked(b):
    with output:
        # Clear previous recordings
        output.clear_output()
        
        # Record audio
        recording, fs = record_audio(duration=5)  # Record for 5 seconds
        
        # Save the recording
        sf.write('recording.wav', recording, fs)
        
        # Display audio widget
        display(Audio('recording.wav', autoplay=False))

# Link button click event to the function
record_button.on_click(on_record_button_clicked)

# Display button
display(record_button, output)

In [ ]:
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Load the recorded audio file
audio_file_path = 'temp.wav'

# Perform speech recognition
with sr.AudioFile(audio_file_path) as source:
    # Record the audio from the file
    audio_data = recognizer.record(source)

    try:
        # Convert speech to text
        text = recognizer.recognize_google(audio_data)
        print("Recognized Text:", text)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")


In [1]:
from google.oauth2 import service_account
from google.cloud import speech

# Provide the path to your service account key file
credentials = service_account.Credentials.from_service_account_file(
    'decoded-bulwark-421920-aa7f61c1c1ad.json'
)

# Pass the credentials to the speech client
client = speech.SpeechClient(credentials=credentials)

def transcribe_local_audio(file_path):
    with open(file_path, "rb") as audio_file:
        content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US"
    )

    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))

# Example: Call the function with your local audio file
transcribe_local_audio('temp.wav')

Transcript: what do you think about Beethoven


In [7]:
from google.cloud import texttospeech
from google.oauth2 import service_account
from pydub import AudioSegment
from pydub.playback import play
import io

# Initialize the Text-to-Speech client
client = texttospeech.TextToSpeechClient(credentials=credentials)

def text_to_speech_and_play(text):
    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request, select the language code and the ssml voice gender
    voice = texttospeech.VoiceSelectionParams(
        language_code='en-US',
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request on the text input with the selected voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Convert the response's audio_content (binary) into an audio segment
    audio_segment = AudioSegment.from_file(io.BytesIO(response.audio_content), format="mp3")

    # Play the audio segment out loud
    play(audio_segment)


# Example usage
text_to_speech("Hello, world! This is a test of Google Cloud Text-to-Speech.")

In [ ]:
import pyaudio
import numpy as np

p = pyaudio.PyAudio()

volume = 0.5     # range [0.0, 1.0]
fs = 44100       # sampling rate, Hz, must be integer
duration = 1.0   # in seconds, may be float
f = 440.0        # sine frequency, Hz, may be float

# generate samples, note conversion to float32 array
samples = (np.sin(2*np.pi*np.arange(fs*duration)*f/fs)).astype(np.float32)

# for paFloat32 sample values must be in range [-1.0, 1.0]
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=fs,
                output=True)

# play. May repeat with different volume values (if done interactively) 
stream.write(volume*samples)

stream.stop_stream()
stream.close()

p.terminate()

In [13]:
import openai

# Set your OpenAI API key
openai.api_key = 'key'

def ask_chatgpt(question):
    """
    Send a question to ChatGPT and return the response.
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question},
        ]
    )
    return response.choices[0].message.content

# Assuming 'transcribed_text' contains your speech-to-text result
gpt_response = ask_chatgpt("Hello ChatGPT")
print(gpt_response)

Hello! How can I assist you today?


In [ ]:
import pyaudio
import wave
import webrtcvad
import speech_recognition as sr
from collections import deque

# Constants for audio capture
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = int(RATE * 0.03)  # 10 ms chunk size
RECORD_SECONDS = 7

# Initialize PyAudio, VAD, and Recognizer
audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(0)  # Level of aggressiveness from 0 to 3
recognizer = sr.Recognizer()

def capture_real_time():
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Starting real-time audio capture and speech detection")
    frames = []
    silent_frames = deque(maxlen=10)  # Store last few frames to add if speech starts

    try:
        while True:
            frame = stream.read(CHUNK, exception_on_overflow=False)
            is_speech = vad.is_speech(frame, RATE)

            if is_speech:
                if silent_frames:
                    frames.extend(silent_frames)  # Prepend silence before speech
                    silent_frames.clear()
                frames.append(frame)
            else:
                silent_frames.append(frame)
                if len(frames) > 0:  # Speech has ended, process the captured frames
                    with wave.open("test_output.wav", "wb") as wf:
                        wf.setnchannels(CHANNELS)
                        wf.setsampwidth(audio_interface.get_sample_size(FORMAT))
                        wf.setframerate(RATE)
                        wf.writeframes(b''.join(frames))
                    process_speech(frames)
                    frames = []

    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        stream.stop_stream()
        stream.close()
        audio.terminate()



capture_real_time()

In [ ]:
import pyaudio
import wave
import webrtcvad
import speech_recognition as sr
from collections import deque

# Constants for audio capture
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = int(RATE * 0.03)  # 10 ms chunk size
SILENT_FRAMES_BUFFER_SIZE = 30  # Store more silent frames

# Initialize PyAudio, VAD, and Recognizer
audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(0)  # Level of aggressiveness from 0 to 3
recognizer = sr.Recognizer()

def capture_real_time():
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Starting real-time audio capture and speech detection")
    audio_frames = []
    silent_frames = deque(maxlen=SILENT_FRAMES_BUFFER_SIZE)  # Store last few frames to add if speech starts

    try:
        while True:
            # Audio processing for speech detection
            audio_frame = stream.read(CHUNK, exception_on_overflow=False)
            is_speech = vad.is_speech(audio_frame, RATE)
    
            if is_speech:
                if silent_frames:
                    audio_frames.extend(silent_frames)
                    silent_frames.clear()
                audio_frames.append(audio_frame)
            else:
                silent_frames.append(audio_frame)
                if len(audio_frames) > 0:
                    detected_text = process_speech(audio_frames)
                    audio_frames = []

    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        stream.stop_stream()
        stream.close()
        audio.terminate()

def process_speech(frames):
    # Write frames to a buffer
    buffer = wave.open('temp.wav', 'wb')
    buffer.setnchannels(CHANNELS)
    buffer.setsampwidth(audio.get_sample_size(FORMAT))
    buffer.setframerate(RATE)
    buffer.writeframes(b''.join(frames))
    buffer.close()

    # Use speech recognition on the buffer
    with sr.AudioFile('temp.wav') as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Recognized text: {text}")
        except sr.UnknownValueError:
            pass
        except sr.RequestError as e:
            print(f"Error from speech recognition service: {e}")

capture_real_time()

In [11]:
import pyaudio
import webrtcvad
import speech_recognition as sr

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 320  # This corresponds to 20 ms of audio at 16000 Hz

vad = webrtcvad.Vad(1)  # Moderate aggressiveness
audio_interface = pyaudio.PyAudio()

def listen_and_process():
    stream = audio_interface.open(format=FORMAT, channels=CHANNELS,
                                  rate=RATE, input=True,
                                  frames_per_buffer=CHUNK)
    print("Listening...")

    speech_frames = bytes()  # buffer to collect frames identified as speech

    try:
        while True:
            frame = stream.read(CHUNK, exception_on_overflow=False)  # Read a frame from the stream
            is_speech = vad.is_speech(frame, RATE)  # Check if the frame contains speech

            if is_speech:
                speech_frames += frame  # Accumulate speech frames

            elif speech_frames:
                # When non-speech detected and there are accumulated speech frames
                process_speech(speech_frames)
                speech_frames = bytes()  # Reset speech frame buffer

    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        stream.stop_stream()
        stream.close()
        audio_interface.terminate()

def process_speech(frames):
    # Process the speech frames
    recognizer = sr.Recognizer()
    
    # Create an instance of AudioData
    audio_data = sr.AudioData(frames, RATE, audio_interface.get_sample_size(FORMAT))
    
    try:
        # Recognize speech using the AudioData instance
        text = recognizer.recognize_google(audio_data)
        print(f"Recognized text: {text}")
    except sr.UnknownValueError:
        # print("Speech was not understood.")
        pass
    except sr.RequestError as e:
        print(f"Error from speech recognition service: {e}")

listen_and_process()

Listening...
Recognized text: what do you think of beeth
Stopping...


In [ ]:
def text_to_speech(text):
    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request, select the language code and the voice type
    voice = texttospeech.VoiceSelectionParams(
        language_code='en-US',
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request on the text input with the selected voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # The response's audio_content is binary.
    with open('output.mp3', 'wb') as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

# Example usage
text_to_speech("Hello, world! This is a test of Google Cloud Text-to-Speech."